<a href="https://colab.research.google.com/github/MdMamunSheikh/Nurse-Care-Challenge-2020/blob/main/src/python/Data_labeling_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Read raw data

#####Import necessary libraries and module

#####File path list

In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import pytz
import time
 
pd.set_option("display.max_row", 1000,"display.max_columns",100)

In [ ]:
MAIN_PATH = "/content/drive/My Drive/ML/Nursing care activity recognitions challenge 2020/Datasets/Field/raw_field_acc_user"
field_users = ["04","07","08","18","38","51"]
field_label = "/content/drive/My Drive/ML/Nursing care activity recognitions challenge 2020/Datasets/Field/field_label_train.csv"
lab_users = "/content/drive/My Drive/ML/Nursing care activity recognitions challenge 2020/Datasets/Lab/bigact_raw_lab_acc.csv"
lab_label = "/content/drive/My Drive/ML/Nursing care activity recognitions challenge 2020/Datasets/Lab/labels_lab_2users.csv"

#####Function defination to read csv files

In [ ]:
#function to read csv formatted file with additional features
def read_file(f):
#read raw file
  tic = time.time()
  s = time.time()
  df = pd.read_csv(f)
  e = time.time()
  print("Time Elapsed to read csv file:", e-s, "seconds\n") 
  print("Raw data size before dropping N/A:",df.shape,"\n")
#drop N/A and duplicate rows
  s = time.time()
  df.dropna(inplace = True)
  print("Raw data size after dropping N/A:",df.shape,"\n")
  df.drop_duplicates(inplace = True)
  print("Raw data size after dropping duplicate rows:",df.shape,"\n")
  e = time.time()
  print("Time Elapsed to drop N/A and duplicate rows:", e-s, "seconds\n")
 
  try :
#datetime conversion
    s = time.time()
    df["datetime"] = pd.to_datetime(df["datetime"], utc = True)
    e = time.time()
    print("Time Elapsed to datetime conversion:", e-s, "seconds\n")
#sorting according to datetime column
    s = time.time()
    df = df.sort_values(by = "datetime", ascending = True)
    e = time.time()
    print("Time Elapsed to sort dataframe:", e-s, "seconds\n")
    toc = time.time()
    print("Total Time:",toc -tic, "seconds\n\n\n\n")
#return dataframe
    return df
            
  except Exception as err:
#datetime conversion
    s = time.time()
    df["start"] = pd.to_datetime(df["start"], utc = True)
    df["finish"] = pd.to_datetime(df["finish"], utc = True)
    e = time.time()
    print("Time Elapsed to datetime conversion:", e-s, "seconds\n")
#sorting according to start column
    s = time.time()
    df  = df.sort_values(by = "start", ascending = True)
    e = time.time()
    print("Time Elapsed to sort dataframe:", e-s, "seconds\n")
    toc = time.time()
    print("Total Time:",toc -tic, "seconds\n\n\n\n")
#returm dataframe
    return df

#####Load datasets

In [ ]:
print("Field label data loading...")
f_label = read_file(field_label)
print("Lab users data loading...")
l_users = read_file(lab_users)
print("Lab label data loading...")
l_label = read_file(lab_label)

Field label data loading...
Time Elapsed to read csv file: 0.012775182723999023 seconds

Raw data size before dropping N/A: (1477, 4) 

Raw data size after dropping N/A: (1477, 4) 

Raw data size after dropping duplicate rows: (1477, 4) 

Time Elapsed to drop N/A and duplicate rows: 0.008674860000610352 seconds

Time Elapsed to datetime conversion: 0.06871795654296875 seconds

Time Elapsed to sort dataframe: 0.0012733936309814453 seconds

Total Time: 0.09249591827392578 seconds




Lab users data loading...
Time Elapsed to read csv file: 1.0240652561187744 seconds

Raw data size before dropping N/A: (1064483, 5) 

Raw data size after dropping N/A: (1064483, 5) 

Raw data size after dropping duplicate rows: (617479, 5) 

Time Elapsed to drop N/A and duplicate rows: 0.4566478729248047 seconds

Time Elapsed to datetime conversion: 10.058229923248291 seconds

Time Elapsed to sort dataframe: 0.11320972442626953 seconds

Total Time: 11.653942108154297 seconds




Lab label data loading...
Ti

In [ ]:
user = {}
st = time.time()
for i in field_users:
  print("user"+i,": Reading file...")
  user[i] = read_file(MAIN_PATH+i+".csv")
et = time.time()
print("Total time to read field users datasets:",et-st,"seconds\n")

user04 : Reading file...
Time Elapsed to read csv file: 17.94185709953308 seconds

Raw data size before dropping N/A: (12625774, 5) 

Raw data size after dropping N/A: (12625774, 5) 

Raw data size after dropping duplicate rows: (8720962, 5) 

Time Elapsed to drop N/A and duplicate rows: 14.606249570846558 seconds

Time Elapsed to datetime conversion: 141.51723504066467 seconds

Time Elapsed to sort dataframe: 1.5204291343688965 seconds

Total Time: 175.58749437332153 seconds




user07 : Reading file...
Time Elapsed to read csv file: 22.305169582366943 seconds

Raw data size before dropping N/A: (15716507, 5) 

Raw data size after dropping N/A: (15716507, 5) 



#####Shapes of the datasets

In [ ]:
print("Field label data size:",f_label.shape)
print("Lab user data size:",l_users.shape)
print("Lab label data size:",l_label.shape)

In [ ]:
for i in field_users:
  print("user"+i+" data size",user[i].shape)

###Datasets labelling

#####Function defination to merge dataframe

In [ ]:
#function to merge lab and field datasets
def merge_df(users, label) :
#make sure that both df have to same user_id
  st = time.time()
  lists = []
  for id in users.keys():
    print("user"+id+" processing...\n")
    s = time.time()
    mask = label["user_id"] == users[id].loc[0, "user_id"]
    new_df = label.loc[mask]
    l = len(new_df)
    new_df.index = range(l)
    X,Y,Z,T =[],[],[],[]
    #print(new_df)
 
    for i in range(l) :
      mask = (users[id].datetime >= new_df.loc[i,"start"]) & (users[id].datetime <= new_df.loc[i,"finish"])
      df = users[id].loc[mask]
    #  print ("df.shape",df.shape)
      if df.shape[0] == 0 :
        new_df.drop([i], inplace = True)
      else:
        X.append(df["x"].to_numpy(dtype = np.float32).reshape(-1,1))
        Y.append(df["y"].to_numpy(dtype = np.float32).reshape(-1,1))
        Z.append(df["z"].to_numpy(dtype = np.float32).reshape(-1,1))
        T.append(df["datetime"].to_numpy().reshape(-1,1))
    
    new_df["X"] = X
    new_df["Y"] = Y
    new_df["Z"] = Z
    new_df["T"] = T
    new_df.index = range(len(new_df.index))
 
    e = time.time()
    print("Time Elapsed to label user" + id + ":", e-s, "sec\n")
    print("Size of user" + id + " before labelling:", l," \n")
    print("Size of user" + id + " after labelling:", len(new_df),"\n\n\n")
 
    lists.append(new_df)
 
  labelled_df = pd.concat(lists, ignore_index = True)
  print("Final labelled data size:",labelled_df.shape," \n")
  et = time.time()
  print("Total time to label datasets:",et - st,"seconds\n\n\n")
 
  return labelled_df

#####Labelling datasets

######Field data labelling

In [ ]:
field_labelled = merge_df(user,f_label)

In [ ]:
field_labelled.info()

######Lab data labelling

In [ ]:
#Don't run this cell twice.
#add 59 seconds to "finish" column
l_label.finish = l_label.finish + pd.Timedelta(seconds = 59.9999)
l_label.head(10)

In [ ]:
 
#make a dictionary based on lab data
l_user01 = l_users.loc[l_users.user_id == 1]
l_user01.index = range(len(l_user01))
l_user19 = l_users.loc[l_users.user_id == 19]
l_user19.index = range(len(l_user19))
lab_users = {"01":l_user01, "19":l_user19}
lab_users

In [ ]:
lab_labelled = merge_df(lab_users,l_label)

######Information about field and lab data

In [ ]:
field_labelled.info()

In [ ]:
lab_labelled.info()

#####Save the labelled dataframe as pkl format

Best compression parameters is zip

######Save and reload field data

In [ ]:
PATH = "/content/drive/My Drive/ML/Nursing care activity recognitions challenge 2020/Datasets/Labelled data/"
field_labelled.to_pickle(PATH+"labelled_field_data.pkl", compression = "zip")

In [ ]:
del field_labelled

In [ ]:
field_labelled = pd.read_pickle(PATH + "labelled_field_data.pkl", compression = "zip")
field_labelled.info()

######Save and reload lab data

In [ ]:
lab_labelled.to_pickle(PATH + "labelled_lab_data.pkl", compression = "zip")

In [ ]:
del lab_labelled

In [ ]:
lab_labelled = pd.read_pickle(PATH + "labelled_lab_data.pkl", compression = "zip")
lab_labelled.info()

In [ ]:
user = pd.read_csv(lab_users)
user.head(10)

In [ ]:
l_users.tail(1000)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_pickle('/content/drive/MyDrive/ML/Nursing care activity recognitions challenge 2020/Datasets/Labelled data/Field.pkl', compression = 'gzip')
df

,user_id,act_id,s,f,x,y,z,t,period
0,4,5,2018-12-02 10:27:37+09:00,2018-12-02 10:28:22+09:00,"[1.3019999999999998, 5.2860000000000005, 3.332...","[8.619, 8.121, 5.707000000000001, 7.125, 7.047...","[2.872, 0.027999999999999997, 7.39, 7.153, 0.2...","[2018-12-02 10:27:57.262000+09:00, 2018-12-02 ...",0 days 00:00:45
1,4,5,2018-12-02 10:32:58+09:00,2018-12-02 10:33:29+09:00,"[2.26, 1.915, 4.06, 8.466000000000001, 8.083, ...","[9.385, 8.351, 8.466000000000001, 4.022, 3.293...","[4.059, 3.514, 0.35700000000000004, -0.401, -2...","[2018-12-02 10:33:07.177000+09:00, 2018-12-02 ...",0 days 00:00:31
2,4,4,2018-12-02 10:33:31+09:00,2018-12-02 10:34:25+09:00,"[1.072, 2.26, 0.038, 0.8420000000000001, -0.38...","[9.577, 8.504, 10.764000000000001, 11.147, 9.3...","[1.851, 3.022, 0.099, 0.408, -0.068, 0.113, 1....","[2018-12-02 10:33:45.773000+09:00, 2018-12-02 ...",0 days 00:00:54
3,4,5,2018-12-02 10:34:29+09:00,2018-12-02 10:35:09+09:00,"[5.631, 3.409, 0.038, 4.06, 3.141, 7.7, 2.7960...","[9.04, 9.423, 10.075, 8.274, 8.083, 4.865, 9.6...","[0.064, 0.08, -0.314, 0.242, 0.309, 6.34399999...","[2018-12-02 10:34:42.184000+09:00, 2018-12-02 ...",0 days 00:00:40
4,4,4,2018-12-02 11:49:03+09:00,2018-12-02 11:50:02+09:00,"[-5.478, -3.792, 1.838, -0.996, -1.87699999999...","[11.684000000000001, 7.355, 9.193999999999999,...","[0.068, 0.16, 0.275, 0.32799999999999996, -0.0...","[2018-12-02 11:49:52.466000+09:00, 2018-12-02 ...",0 days 00:00:59
...,...,...,...,...,...,...,...,...,...
933,51,12,2018-12-28 18:50:15+09:00,2018-12-28 18:50:23+09:00,"[8.619, -1.072, 7.278, 7.316, 6.397, 9.921, 6....","[5.861000000000001, 9.577, 7.776, 5.3629999999...","[-0.05, 0.03, 0.319, 3.8480000000000003, -0.30...","[2018-12-28 18:50:20.870000+09:00, 2018-12-28 ...",0 days 00:00:08
934,51,12,2018-12-29 18:02:04+09:00,2018-12-29 18:02:14+09:00,"[6.704, 2.068, 5.3629999999999995, 1.646999999...","[5.784, 10.725999999999999, 7.355, 11.645, 8.5...","[0.855, 0.387, 0.392, 0.005, 0.033, 2.805, 0.3...","[2018-12-29 18:02:11.292000+09:00, 2018-12-29 ...",0 days 00:00:10
935,51,12,2018-12-29 18:11:36+09:00,2018-12-29 18:11:42+09:00,"[0.804, 6.359, 3.0260000000000002, 2.796000000...","[8.772, 7.622999999999999, 8.964, 8.274, 11.30...","[2.5269999999999997, 0.098, 0.3989999999999999...","[2018-12-29 18:11:38.237000+09:00, 2018-12-29 ...",0 days 00:00:06
936,51,12,2018-12-29 18:19:57+09:00,2018-12-29 18:20:07+09:00,"[7.125, -1.6469999999999998, 5.171, 8.427, 0.0...","[9.117, 9.462, 8.542, 7.507999999999999, 9.423...","[2.6910000000000003, 3.1060000000000003, 0.078...","[2018-12-29 18:20:04.183000+09:00, 2018-12-29 ...",0 days 00:00:10
